## アプリケーション動作確認

In [22]:
# モジュール
from modules.models.models import *
from modules import spotify_tools as st
from app import create_app


In [23]:
# app定義
app = create_app()
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///..\\..\\data\\user_database.db"


### SQLAlcemy動作確認
- https://flask-sqlalchemy.palletsprojects.com/en/3.1.x/queries/
- https://flask-sqlalchemy.palletsprojects.com/en/3.1.x/legacy-query/

In [24]:
# db start db = SQLAlchemy()はmodule内で定義されているので重複を避ける
db.init_app(app)
app.app_context().push()

In [25]:
# db作成
db.create_all()

2024-03-02 20:55:10,211 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-02 20:55:10,214 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2024-03-02 20:55:10,215 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-02 20:55:10,218 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("fav_artists")
2024-03-02 20:55:10,219 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-02 20:55:10,224 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("sim_users")
2024-03-02 20:55:10,225 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-02 20:55:10,229 INFO sqlalchemy.engine.Engine COMMIT


In [28]:
# user_listの作成
user_list = UserList()
# add
user_list.add_user(user_token="#letseat", display_name="chika", pass_word="eat") 
# assert db.session.query(User).filter_by(user_token="#thisistoken").first(), "Userオブジェクトが作成されていません"


2024-03-02 20:55:36,275 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-02 20:55:36,277 INFO sqlalchemy.engine.Engine INSERT INTO users (user_token, user_name, pass_word, done) VALUES (?, ?, ?, ?)
2024-03-02 20:55:36,278 INFO sqlalchemy.engine.Engine [cached since 24.69s ago] ('#letseat', 'chika', 'eat', 0)
2024-03-02 20:55:36,281 INFO sqlalchemy.engine.Engine COMMIT


In [27]:
# errorが出た際にセッション
db.session.rollback()
user_list.varified = True
db.session.commit()

In [29]:
# check UserList
user = user_list.get_user_info("#letseat")

# 要素の取得
# name 
user_name = user.user_name
# password
user_pass = user.pass_word
# token
user_token = user.user_token

print("\n")
print(f"user_name: {user_name}")
print(f"user_pass: {user_pass}")
print(f"user_token: {user_token}")


2024-03-02 20:55:48,211 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-02 20:55:48,214 INFO sqlalchemy.engine.Engine SELECT users.user_token AS users_user_token, users.user_name AS users_user_name, users.pass_word AS users_pass_word, users.done AS users_done 
FROM users 
WHERE users.user_token = ?
 LIMIT ? OFFSET ?
2024-03-02 20:55:48,215 INFO sqlalchemy.engine.Engine [generated in 0.00127s] ('#letseat', 1, 0)


user_name: chika
user_pass: eat
user_token: #letseat


In [44]:
# 404エラー受け取り
try:
    # 例外発生受け取り
    c = User.query.filter_by(user_name="amazingman").first_or_404()
    print(c)
except BaseException as e:
    print("で、でた～", e)


2024-03-02 21:03:15,970 INFO sqlalchemy.engine.Engine SELECT users.user_token AS users_user_token, users.user_name AS users_user_name, users.pass_word AS users_pass_word, users.done AS users_done 
FROM users 
WHERE users.user_name = ?
 LIMIT ? OFFSET ?
2024-03-02 21:03:15,971 INFO sqlalchemy.engine.Engine [cached since 342.3s ago] ('amazingman', 1, 0)
<User #wowow>


In [47]:
# 存在可否
c = User.query.filter_by(user_name="foke").first()
print(c)

2024-03-02 21:05:48,335 INFO sqlalchemy.engine.Engine SELECT users.user_token AS users_user_token, users.user_name AS users_user_name, users.pass_word AS users_pass_word, users.done AS users_done 
FROM users 
WHERE users.user_name = ?
 LIMIT ? OFFSET ?
2024-03-02 21:05:48,337 INFO sqlalchemy.engine.Engine [cached since 494.6s ago] ('foke', 1, 0)
None


In [52]:
# db終了
db.session.commit()
db.session.close()



### アプリケーション動作

In [1]:
from flask import Flask, request, jsonify
from modules.models.models import *


In [5]:
# app setting
def create_app(config="base"):
    # instance_relative_config : 相対的なconfigファイルを指定
    app = Flask(__name__, instance_relative_config=True)
    app.config.from_mapping(
        SECRET_KEY="test_secret"
    )

    # configファイル : https://qiita.com/nanakenashi/items/e272ff1aafb3889230bc
    if config == "test":
        app.config.from_object("config.TestingConfig")
    elif config == "development":
        app.config.from_object("config.DevelopmentConfig")
    elif config == "base":
        app.config.from_object("config.BaseConfig")
    
    # SQLAlchemy
    db.init_app(app)
    app.app_context().push()
    db.create_all()
    user_list = UserList()

    @app.route("/v1/get_user_data", methods=["POST"])
    def get_user_data():
        return
    
    @app.route("/v1/regist_user_data", methods=["POST"])
    def regist_user_data():
        """
        Summary:
            新規ユーザの登録.
        Args:
            display_name: ユーザ名
            pass_word: パスワード
        """
        # リクエスト受け取り
        # JSONを受け取る
        json = request.get_json()
        display_name = json["display_name"]
        pass_word = json["pass_word"]
        
        # 既存である場合　登録済みエラー
        is_known = User.query.filter_by(user_name=display_name).one_or_404()
        if is_known != None:
            return jsonify({"user_id": None})
        
        pass_word = pass_word
        user_id = "#good"

        # 登録
        user_list.add_user(
            user_token=user_id,
            display_name=display_name,
            pass_word=pass_word
        )

        return jsonify({"user_id": user_id})

    @app.route("/v1/delete_user_data", methods=["POST"])
    def delete_user_data():
        return

    def make_artist_data():
        return

    def find_similar_user():
        return
    
    return app



2024-03-02 21:24:26,078 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-02 21:24:26,079 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2024-03-02 21:24:26,080 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-02 21:24:26,082 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("fav_artists")
2024-03-02 21:24:26,084 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-02 21:24:26,088 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("sim_users")
2024-03-02 21:24:26,088 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-02 21:24:26,088 INFO sqlalchemy.engine.Engine COMMIT
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:8888
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

c:\Users\ryotaro\Documents\03_個人_趣味\MyGithub\club_activity_app\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
# テスト
app = create_app(config="development")
app.run(port=8888)


2024-03-02 21:21:54,380 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-02 21:21:54,382 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2024-03-02 21:21:54,384 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-02 21:21:54,386 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("fav_artists")
2024-03-02 21:21:54,386 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-02 21:21:54,386 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("sim_users")
2024-03-02 21:21:54,386 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-02 21:21:54,401 INFO sqlalchemy.engine.Engine COMMIT
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:8888
Press CTRL+C to quit


 * Restarting with stat


SystemExit: 1

c:\Users\ryotaro\Documents\03_個人_趣味\MyGithub\club_activity_app\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [18]:
import requests
import json
data = {
    "display_name":"goodman",
    "pass_word":"goodgood"    
}

content = requests.post("http://127.0.0.1:8888/v1/regist_user_data", data=data)



In [11]:
print(content)

<Response [415]>
